In [ ]:
import numpy as np
import nibabel as nib
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage.transform as skTrans
import math

In [ ]:
rootDirPath = "..\\mmwhs\\ct_train\\"

In [ ]:
def CreateSaggitalUnet(x, y, channel, num_classes=8):

    # [slice, :, :]
    inputs = tf.keras.layers.Input((x, y, channel))
    kernel_initializer = "he_normal"

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    p1 = tf.keras.layers.PReLU()(p1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    p2 = tf.keras.layers.PReLU()(p2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    p3 = tf.keras.layers.PReLU()(p3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = tf.keras.layers.PReLU()(p4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

    #Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.PReLU()(u6)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.PReLU()(u7)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.PReLU()(u8)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.PReLU()(u9)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
def CreateCoronalUnet(x, y, channel, num_classes=8):

    # [:, slice, :]
    inputs = tf.keras.layers.Input((x, y, channel))
    kernel_initializer = "he_normal"

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    p1 = tf.keras.layers.PReLU()(p1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    p2 = tf.keras.layers.PReLU()(p2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    p3 = tf.keras.layers.PReLU()(p3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = tf.keras.layers.PReLU()(p4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

    #Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.PReLU()(u6)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.PReLU()(u7)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.PReLU()(u8)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.PReLU()(u9)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
def CreateAxialUnet(x, y, channel, num_classes=8):
    # [:, :, slice]
    inputs = tf.keras.layers.Input((x, y, channel))
    kernel_initializer = "he_normal"

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    p1 = tf.keras.layers.PReLU()(p1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    p2 = tf.keras.layers.PReLU()(p2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    p3 = tf.keras.layers.PReLU()(p3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = tf.keras.layers.PReLU()(p4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

    #Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.PReLU()(u6)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.PReLU()(u7)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.PReLU()(u8)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.PReLU()(u9)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, num_classes):
    kernel_initializer = 'he_uniform'  # Try others if you want
    # Build the model
    inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    # s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    # Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(s)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c5)

    # Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu',
                kernel_initializer=kernel_initializer, padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
def OneHotEncode2D(original, x_, y_, num_classes=8):
    encodeList = {
        0: 0,
        500: 1,
        600: 2,
        420: 3,
        550: 4,
        205: 5,
        820: 6,
        850: 7,
    }

    x, y = original.shape

    labelEncode = np.empty((x_, y_, num_classes), dtype="float16")
    for i_ in range(x_):
        for j_ in range(y_):
            i = math.floor( (i_ * x) / x_)
            j = math.floor( (j_ * y) / y_)

            i = max(0, min(i, x - 1))
            j = max(0, min(j, y - 1))

            value = original[i][j]

            encodeIndex = encodeList[value]

            for n in range(8):
                labelEncode[i_][j_][n] = 0

            labelEncode[i_][j_][encodeIndex] = 1

    return labelEncode

def ResizeImage(image, x_, y_):
  return np.array(skTrans.resize(image, (x_, y_), order=1, preserve_range=True, anti_aliasing=False), dtype="float16")

In [ ]:
def NormalizeImageIntensityRange(image):
  houndsfieldMax = 2000
  houndsfieldMin = -1000
  image[image < houndsfieldMin] = houndsfieldMin
  image[image > houndsfieldMax] = houndsfieldMax

  # min max intensity normalization with houndsfield
  return (image - houndsfieldMin) / (houndsfieldMax - houndsfieldMin)

In [ ]:
def GenerateData(start, stop, rootDirPath, viewpoint):
  def SortImageKey(filename):
    parts = filename.split('_')
    return (int(parts[2]), parts[3])

  x = []
  y = []
  pathList = sorted(os.listdir(rootDirPath), key=SortImageKey)

  for f in range(start, stop, 2):
    image = np.array(nib.load(os.path.join(rootDirPath, pathList[f])).get_fdata(), dtype="float16")
    label = np.array(nib.load(os.path.join(rootDirPath, pathList[f + 1])).get_fdata(), dtype="float16")
    for i in range(image.shape[viewpoint - 1]):
      if viewpoint == 1:
        newImage = image[i, :, :]
        newImage = ResizeImage(newImage, 256, 256)
        newImage = NormalizeImageIntensityRange(newImage)
        newImage = np.expand_dims(newImage, -1)

        newLabel = label[i, :, :]
        newLabel = OneHotEncode2D(newLabel, 256, 256)

        x.append(newImage)
        y.append(newLabel)

      elif viewpoint == 2:
        newImage = image[:, i, :]
        newImage = ResizeImage(newImage, 256, 256)
        newImage = NormalizeImageIntensityRange(newImage)
        newImage = np.expand_dims(newImage, -1)

        newLabel = label[:, i, :]
        newLabel = OneHotEncode2D(newLabel, 256, 256)

        x.append(newImage)
        y.append(newLabel)

      elif viewpoint == 3:
        newImage = image[:, :, i]
        newImage = ResizeImage(newImage, 256, 256)
        newImage = NormalizeImageIntensityRange(newImage)
        newImage = np.expand_dims(newImage, -1)

        newLabel = label[:, :, i]
        newLabel = OneHotEncode2D(newLabel, 256, 256)

        x.append(newImage)
        y.append(newLabel)

  x = np.array(x)
  y = np.array(y)
  return x, y

In [ ]:
def GenerateFullResData(start, stop, rootDirPath):
  def SortImageKey(filename):
    parts = filename.split('_')
    return (int(parts[2]), parts[3])

  x = []
  y = []
  pathList = sorted(os.listdir(rootDirPath), key=SortImageKey)

  for f in range(start, stop, 2):
    image = np.array(nib.load(os.path.join(
        rootDirPath, pathList[f])).get_fdata(), dtype="float16")
    label = np.array(nib.load(os.path.join(
        rootDirPath, pathList[f + 1])).get_fdata(), dtype="float16")
    for i in range(image.shape[-1]):
        
        newImage = image[:, :, i]
        newImage = NormalizeImageIntensityRange(newImage)
        newImage = np.expand_dims(newImage, -1)

        newLabel = label[:, :, i]
        newLabel = OneHotEncode2D(newLabel, 512, 512)

        x.append(newImage)
        y.append(newLabel)

  x = np.array(x)
  y = np.array(y)
  return x, y

In [ ]:
x_axial_full_res, y_axial_full_res = GenerateFullResData(0, 30, rootDirPath)

In [ ]:
x = np.load("./x_axial.npy")
print(x.shape)

In [ ]:
y = np.load("./y_axial.npy")
print(y.shape)

In [ ]:
def TrainingLoop(model, originalX, originalY):
  x = []
  y = []
  trainingHistory = {}

  for i in range(0, 15, 5):
    x.append(originalX[i])
    x.append(originalX[i + 1])
    x.append(originalX[i + 2])
    x.append(originalX[i + 3])
    x.append(originalX[i + 4])

    y.append(originalY[i])
    y.append(originalY[i + 1])
    y.append(originalY[i + 2])
    y.append(originalY[i + 3])
    y.append(originalY[i + 4])

    x = np.array(x)
    y = np.array(y)

    print(f"Image: {x.shape}")
    print(f"Label: {y.shape}")

    history = model.fit(x, y, epochs=3, validation_split=0.25)
    for key, value in history.history.items():
      if key not in trainingHistory:
          trainingHistory[key] = []
      trainingHistory[key].extend(value)

    x = []
    y = []

  return trainingHistory

In [ ]:
def TrainingLoopWithEvaluate(model, originalX, originalY, validateX, validateY):
  x = []
  y = []
  trainingHistory = {}

  early_stopping = tf.keras.callbacks.EarlyStopping(
  monitor='val_accuracy',
  patience=5,
  restore_best_weights=True
  )

  for i in range(0, 15, 5):
    
    x.append(originalX[i])
    x.append(originalX[i + 1])
    x.append(originalX[i + 2])
    x.append(originalX[i + 3])
    x.append(originalX[i + 4])

    y.append(originalY[i])
    y.append(originalY[i + 1])
    y.append(originalY[i + 2])
    y.append(originalY[i + 3])
    y.append(originalY[i + 4])

    x = np.array(x)
    y = np.array(y)

    print(f"Image: {x.shape}")
    print(f"Label: {y.shape}")

    history = model.fit(x, y, epochs=5, validation_data=(validateX, validateY), callbacks=[early_stopping])
    for key, value in history.history.items():
      if key not in trainingHistory:
          trainingHistory[key] = []
      trainingHistory[key].extend(value)

    x = []
    y = []

  return trainingHistory

In [ ]:
xValidate = np.load("./x_axialValidate.npy")
yValidate = np.load("./y_axialValidate.npy")

In [ ]:
unetSaggital = CreateSaggitalUnet(256, 256, 1)

In [ ]:
trainingHistory = TrainingLoop(unetSaggital, x, y)

In [ ]:
unetSaggital.save('.\\trained_model\\unetSaggital.h5', save_format='h5')

In [ ]:
plt.plot(trainingHistory['loss'])
plt.plot(trainingHistory['val_loss'])
plt.title('Saggital Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


In [ ]:
plt.plot(trainingHistory['accuracy'])
plt.plot(trainingHistory['val_accuracy'])
plt.title('Saggital Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()


In [ ]:
unetCoronal = CreateCoronalUnet(256, 256, 1)

In [ ]:
trainingHistoryCoronal = TrainingLoop(unetCoronal, x, y)

In [ ]:
plt.plot(trainingHistoryCoronal['loss'])
plt.plot(trainingHistoryCoronal['val_loss'])
plt.title('Coronal Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


In [ ]:
plt.plot(trainingHistoryCoronal['accuracy'])
plt.plot(trainingHistoryCoronal['val_accuracy'])
plt.title('Coronal Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()


In [ ]:
unetCoronal.save('.\\trained_model\\unetCoronal.h5', save_format='h5')

In [ ]:
xValidate = np.load("./x_axialValidate.npy")
yValidate = np.load("./y_axialValidate.npy")

In [ ]:
unetAxial = CreateAxialUnet(256, 256, 1)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
monitor='val_accuracy',
patience=5,
restore_best_weights=True
)

unetAxial.fit(x, y, validation_data=(xValidate, yValidate), callbacks=[early_stopping], epochs=20)

In [ ]:
trainingHistoryAxial = TrainingLoopWithEvaluate(unetAxial, x, y, xValidate, yValidate)

In [ ]:
plt.plot(trainingHistoryAxial['loss'])
plt.plot(trainingHistoryAxial['val_loss'])
plt.title('Axial Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


In [ ]:
plt.plot(trainingHistoryAxial['accuracy'])
plt.plot(trainingHistoryAxial['val_accuracy'])
plt.title('Axial Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()


In [ ]:
unetAxial.save('.\\trained_model\\unetAxial.h5', save_format='h5')